In [134]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
# import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import pandas as pd, numpy as np
import matplotlib as mpl
# import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier


In [135]:
Scientific_Sample=pd.read_csv("/Users/wangxiaoyan/Documents/wy/论文/git/scientific-worker-mental-health/data/all/2017科技人员心理健康调查原始数据.csv")


/var/folders/6r/np24csk96nn1jm3ckkjcn48r0000gq/T/ipykernel_90963/2620090110.py:1: DtypeWarning: Columns (319) have mixed types. Specify dtype option on import or set low_memory=False.
  Scientific_Sample=pd.read_csv("/Users/wangxiaoyan/Documents/wy/论文/git/scientific-worker-mental-health/data/all/2017科技人员心理健康调查原始数据.csv")


In [136]:
#正性与负性情绪量表
Scientific_Sample.drop(columns=Scientific_Sample.filter(regex='^PANAS').columns, inplace=True)


In [137]:
cesd_columns = [col for col in Scientific_Sample.columns if col.startswith('CESD')]
anxiety_columns = [col for col in Scientific_Sample.columns if col.startswith("GAD")]
selected_columns = ['suicide']

for column in selected_columns:
    # 计算众数
    mode_value = Scientific_Sample[column].mode().iloc[0]
    # 使用众数填充空值
    Scientific_Sample[column].fillna(mode_value, inplace=True)

for column in anxiety_columns:
    # 计算众数
    mode_value = Scientific_Sample[column].mode().iloc[0]
    # 使用众数填充空值
    Scientific_Sample[column].fillna(mode_value, inplace=True)
for column in cesd_columns:
    # 计算众数
    mode_value = Scientific_Sample[column].mode().iloc[0]
    # 使用众数填充空值
    Scientific_Sample[column].fillna(mode_value, inplace=True)


In [138]:
CESD12 = 'CESD12'  # 将这里替换为你需要修改的列名
CESD16 = 'CESD16'  # 将这里替换为你需要修改的列名
old_value1 = '0'  # 将这里替换为需要替换的第一个值
new_value1 = '3'  # 将这里替换为替换后的第一个新值
old_value2 = '1'  # 将这里替换为需要替换的第二个值
new_value2 = '2'  # 将这里替换为替换后的第二个新值

# 创建一个字典，其中键是要替换的旧值，值是要替换的新值
replacement_dict = {old_value1: new_value1, old_value2: new_value2}
# 执行替换
Scientific_Sample[CESD12] = Scientific_Sample[CESD12].replace(replacement_dict)
Scientific_Sample[CESD16] = Scientific_Sample[CESD16].replace(replacement_dict)


depression_columns = [col for col in Scientific_Sample.columns if col.startswith("CESD")]
depression_sum = Scientific_Sample[depression_columns].sum(axis=1)
Scientific_Sample['depression'] = depression_sum

In [139]:
Scientific_Sample['depression'] = pd.cut(Scientific_Sample.depression, [-1, 9, 16,np.inf], labels=[0, 1, 2])


In [140]:
anxiety_columns = [col for col in Scientific_Sample.columns if col.startswith("GAD")]
anxiety_sum = Scientific_Sample[anxiety_columns].sum(axis=1)
Scientific_Sample['anxiety'] = anxiety_sum

In [141]:
Scientific_Sample['anxiety'] = pd.cut(Scientific_Sample.anxiety, [-1, 4, 9, 14,np.inf], labels=[0, 1, 2, 3])


In [142]:
Scientific_Sample.drop(columns=Scientific_Sample.filter(regex='^GAD').columns, inplace=True)
Scientific_Sample.drop(columns=Scientific_Sample.filter(regex='^CESD').columns, inplace=True)
print("行数:", Scientific_Sample.shape[0])


行数: 13284


In [143]:
Scientific_Sample = Scientific_Sample.dropna(axis = 0,subset =['suicide'])
Scientific_Sample['suicide'] = pd.cut(Scientific_Sample.suicide, [-1,3,np.inf], labels=[0, 1])
print("行数:", Scientific_Sample.shape[0])


行数: 13284


In [144]:
cols = Scientific_Sample.columns.tolist()

# 将需要移动的列从原来的位置弹出
col_to_move = 'suicide'  # 替换为需要移动的列的名称
cols.pop(cols.index(col_to_move))
# 将移动的列添加到列列表的末尾
cols.append(col_to_move)
# 重新排列 DataFrame 中的列
Scientific_Sample = Scientific_Sample[cols]


In [145]:
for column in Scientific_Sample.columns:
    # 找到列的众数
    mode = Scientific_Sample[column].mode().iloc[0]
    
    # 用众数填充空值
    Scientific_Sample[column].fillna(mode, inplace=True)


In [146]:
background_columns = Scientific_Sample[['RELIGIOUSB','Politicalstate','EduBG','major','Post1','workyear1','WorkingS','ProfessionalRT','TMIncome','NETIncome','AFECQ1','MaritalS','MAT1','NumberCH','NumberCH7','height','weight','exenumweek','exeminp','SMOKING1','SMOKING3','DRINKING1','EPQas47','EPQas30','EPQas12','casedup1','casedup2','Age','Agegrp1','Gender','PSQI1','PSQI4','WORKTIME1','WORKTIME4','Yincome','depression','anxiety','suicide']]

# 将选定的列保存到另一个CSV文件中
background_columns.to_csv("/Users/wangxiaoyan/Documents/wy/论文/git/scientific-worker-mental-health/data/all/2017科技人员心理健康-背景信息数据.csv", index=False)


In [133]:
Scientific_Sample = Scientific_Sample.drop(labels=['ID','NumberCH7','STPQpe17','STPQlt12','CSDMHS17','STPQai10','Irritability','Group','dtot$$$','START_TIME','END_TIME','USED_TIME','受访者-职业群体标签1','interviewee-Occupational group label1','受访者-职业群体标签2','interviewee-Occupational group label2','interviewee-Occupational group label3','受访者-年龄群体标签','Age group label1','Age group label2','ResY','TOKEN','A0','STATUS','QPW'],axis=1)
Scientific_Sample = Scientific_Sample.drop(labels=['Yincome','DOBy','EduBG','Post1','workyear1','WorkingS','TMIncome','NETIncome','AFECQ1','MAT1','NumberCH','height','weight','exenumweek','exeminp','SMOKING1','SMOKING3','DRINKING1','EPQas47','EPQas30','EPQas12','Station','casedup1','casedup2','Age','Agegrp1','Gender','RELIGIOUSB','Politicalstate','major','ProfessionalRT','Post2','MaritalS','WORKTIME1','WORKTIME4'],axis=1)

Scientific_Sample.to_csv("/Users/wangxiaoyan/Documents/wy/论文/git/scientific-worker-mental-health/data/all/2017科技人员心理健康-空值填充处理后数据.csv",sep=',',index=False)
